# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,beroroha,-21.6667,45.1667,29.22,43,66,4.83,MG,1701712279
1,1,grytviken,-54.2811,-36.5092,1.70,63,55,6.17,GS,1701712147
2,2,nur-sultan,51.1801,71.4460,-3.03,80,100,7.00,KZ,1701712280
3,3,portland,45.5234,-122.6762,16.30,91,100,7.60,US,1701711660
4,4,papatowai,-46.5619,169.4708,5.94,78,1,0.82,NZ,1701712167


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "EsriImagery",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    alpha = 0.75,
    color = "City")

# Display the map
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city = []
for index, row in city_data_df.iterrows():
    max_temp = city_data_df.loc[index, 'Max Temp']
    wind = city_data_df.loc[index, "Wind Speed"]
    cloud = city_data_df.loc[index, "Cloudiness"]
    if (max_temp < 27) & (max_temp > 19):
        if wind < 5:
            if cloud == 0:
                ideal_city.append(city_data_df.loc[index])
ideal_city = pd.DataFrame(ideal_city)

# Drop any rows with null values
ideal_drop_na = ideal_city.dropna(axis=0, how="any")

# Display sample data
ideal_drop_na

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,hawaiian paradise park,19.5933,-154.9731,21.69,93,0,3.60,US,1701712169
131,131,fort bragg,35.1390,-79.0060,19.39,43,0,3.58,US,1701712177
161,161,as sulayyil,20.4607,45.5779,20.14,36,0,3.98,SA,1701712209
195,195,ha'il,27.5219,41.6907,20.01,42,0,3.09,SA,1701712305
205,205,orosi,36.5449,-119.2873,19.53,71,0,0.45,US,1701712306
207,207,chibuto,-24.6867,33.5306,22.84,64,0,4.38,MZ,1701712306
226,226,lompoc,34.6391,-120.4579,19.77,79,0,3.60,US,1701712045
275,275,tamanrasset,22.7850,5.5228,20.95,11,0,3.09,DZ,1701712214
367,367,guerrero negro,27.9769,-114.0611,21.60,35,0,0.56,MX,1701712329
403,403,qurayyat,31.3318,37.3428,19.10,39,0,3.09,SA,1701712336


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_drop_na[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
24,hawaiian paradise park,US,19.5933,-154.9731,93,
131,fort bragg,US,35.1390,-79.0060,43,
161,as sulayyil,SA,20.4607,45.5779,36,
195,ha'il,SA,27.5219,41.6907,42,
205,orosi,US,36.5449,-119.2873,71,
207,chibuto,MZ,-24.6867,33.5306,64,
226,lompoc,US,34.6391,-120.4579,79,
275,tamanrasset,DZ,22.7850,5.5228,11,
367,guerrero negro,MX,27.9769,-114.0611,35,
403,qurayyat,SA,31.3318,37.3428,39,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 10

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
as sulayyil - nearest hotel: No hotel found
ha'il - nearest hotel: فندق كلاسيك
orosi - nearest hotel: No hotel found
chibuto - nearest hotel: DESHENG
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
tamanrasset - nearest hotel: فندق الأمان
guerrero negro - nearest hotel: Plaza sal paraiso
qurayyat - nearest hotel: دانة الشمال
nsanje - nearest hotel: No hotel found
chimoio - nearest hotel: Hotel Amirana
marianna - nearest hotel: Executive Inn
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
ovalle - nearest hotel: Hotel Roxy
frontera - nearest hotel: Quirino


,City,Country,Lat,Lng,Humidity,Hotel Name
24,hawaiian paradise park,US,19.5933,-154.9731,93,No hotel found
131,fort bragg,US,35.1390,-79.0060,43,Airborne Inn Lodging
161,as sulayyil,SA,20.4607,45.5779,36,No hotel found
195,ha'il,SA,27.5219,41.6907,42,فندق كلاسيك
205,orosi,US,36.5449,-119.2873,71,No hotel found
207,chibuto,MZ,-24.6867,33.5306,64,DESHENG
226,lompoc,US,34.6391,-120.4579,79,Embassy Suites by Hilton Lompoc Central Coast
275,tamanrasset,DZ,22.7850,5.5228,11,فندق الأمان
367,guerrero negro,MX,27.9769,-114.0611,35,Plaza sal paraiso
403,qurayyat,SA,31.3318,37.3428,39,دانة الشمال


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "EsriImagery",
    frame_width =650,
    frame_height = 500,
    alpha = 0.75,
    color = "City",
    hover_cols = ["Hotel Name","Country"],
    size = 'Humidity',
    )

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)